In [ ]:
import os
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch import autograd
from torch.utils import data
from torch.optim import Adam

import matplotlib.pyplot as plt
import seaborn as sns

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
DATASET_PATH = os.path.join('/content/drive/MyDrive/AI_team/eda')
df_train = pd.read_csv(os.path.join(DATASET_PATH, 'train.csv'))
df_valid = pd.read_csv(os.path.join(DATASET_PATH, 'validate.csv'))

In [ ]:
len(list(df_valid.index))   # valid는 빠짐없이 시간 존재 총 14일 데이터

336

In [ ]:
del_days = 7
df_valid2 = df_valid.drop(df_valid.index[:del_days*24])
df_valid2.to_csv('/content/drive/MyDrive/AI_team/eda/valid2.csv', index=False)

In [ ]:
# train과 valid2 합치기
frames = [df_train, df_valid2]
df_all = pd.concat(frames, ignore_index=True)

In [ ]:
df_all2 = df_all.copy()

In [ ]:
# 결측값 2월6일 18시 / 5월15일 05시
df_all2[df_all2['10']==0]

,날짜,시간,10,100,101,120,121,140,150,160,200,201,251,270,300,301,351,352,370,400,450,500,550,600,650,652,1000,1020,1040,1100,1200,1510,2510,3000,4510,5510,6000
882,20200206,18,0,86,0,1,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,5,49,0,0,4,0,0,0,35,0,0
3212,20200515,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
# 전체 도로 결측치 채워주기
col_list = list(df_all2.columns)
road_list = col_list[2:]

for road in road_list:
    df_all2.loc[882, road] = int( (df_all2.loc[881,road] + df_all2.loc[883,road]) / 2)
    df_all2.loc[3212, road] = int( (df_all2.loc[3211,road] + df_all2.loc[3213,road]) / 2)

In [ ]:
df_all2[df_all2['날짜']==20200220]

,날짜,시간,10,100,101,120,121,140,150,160,200,201,251,270,300,301,351,352,370,400,450,500,550,600,650,652,1000,1020,1040,1100,1200,1510,2510,3000,4510,5510,6000
1200,20200220,0,81360,15875,1422,3603,987,926,24188,978,2234,1792,12213,2656,5255,2351,5705,9724,1684,4732,22577,20241,9779,1511,1607,1210,30849,868,3003,7942,5543,1092,3867,424,2095,1780,2506
1201,20200220,1,56446,10651,789,2489,508,719,17086,604,1438,1469,8748,2458,4276,1916,4475,6923,1169,3600,16804,14845,6011,1073,1044,683,19602,550,1972,4874,3591,1027,2746,304,1409,1145,1835
1202,20200220,2,46005,9053,606,2173,421,644,13363,544,1493,1383,7059,2054,3731,1901,4242,6231,992,3134,13352,12052,5414,901,745,786,14168,438,1694,3749,2579,730,2529,282,1182,948,1474
1203,20200220,3,46667,10675,721,2155,375,561,13468,557,1572,2000,7100,2399,4017,1651,4202,6731,1167,4124,13490,12932,5487,981,895,734,13711,369,1854,4188,2397,721,2463,242,1423,984,1778
1204,20200220,4,60795,17493,1187,3335,662,712,19226,988,2130,2268,9079,3296,5676,2258,5827,8222,1592,6442,19604,19984,8321,1370,1542,1628,21453,541,2693,6758,3492,822,3207,423,2787,1710,2811
1205,20200220,5,111103,32949,1987,6118,1701,1262,44369,1752,3324,2696,15492,4717,8103,3817,8706,16267,2744,11309,26922,41095,15388,2422,2297,2551,66015,1314,6832,19094,10330,1243,5197,715,4816,3835,6281
1206,20200220,6,207060,61011,4668,9760,4875,2428,73351,3605,6081,3479,29141,6895,10016,7202,13996,29051,4555,19070,33891,68408,32339,3254,4512,5677,129193,4214,14117,36616,19242,2119,8373,1398,13019,8685,13468
1207,20200220,7,305576,97610,8061,17635,9858,4954,100021,5840,10481,5804,54127,9334,14017,13006,23021,51257,6494,29769,43612,102337,58874,5983,8527,9414,174034,9177,19478,48098,25648,3267,17817,3104,24641,14367,23759
1208,20200220,8,322371,102599,9723,22520,11468,6449,114586,6698,12216,7390,67130,11864,18877,16176,29816,62770,8569,33936,50474,119822,69138,9144,13482,8664,173321,9189,18109,47826,24601,4158,23138,4357,24022,15044,23778
1209,20200220,9,329382,99455,10884,25087,10588,6574,124913,6661,14504,8330,71054,13801,23254,18636,33490,66119,10044,36426,59973,121133,73923,12895,13638,10070,154277,7652,17055,45315,21247,5434,21828,2826,18934,13957,22361


In [ ]:
# 설날 낀 주 삭제(20200220~20200226) 20200220 인덱스 : 1200
df_all2[df_all2['날짜']==20200220]

# 설날 4일 삭제
df_all3 = df_all2.drop(df_all2.index[1200:1200+7*24])

# 삭제 확인
df_all3[df_all3['날짜']==20200226]

,날짜,시간,10,100,101,120,121,140,150,160,200,201,251,270,300,301,351,352,370,400,450,500,550,600,650,652,1000,1020,1040,1100,1200,1510,2510,3000,4510,5510,6000


In [ ]:
# valid data에서 결측치 없애주기 (5월 21일) 10시~14시 데이터가 이상함 
valid_null_list = ['101', '121', '140']

df_all3[df_all3['101']==0]  # 5월21일 인덱스 3362 이전 주인 14일 값 반영 - 스케일도 별 차이 없음

for road in valid_null_list:
    df_all3.loc[3361, road] = int( (df_all2.loc[3361-7*24, road]) )
    df_all3.loc[3362, road] = int( (df_all2.loc[3362-7*24, road]) )
    df_all3.loc[3363, road] = int( (df_all2.loc[3363-7*24, road]) )
    df_all3.loc[3364, road] = int( (df_all2.loc[3364-7*24, road]) )
    df_all3.loc[3365, road] = int( (df_all2.loc[3365-7*24, road]) )


In [ ]:
df_all3[3193:3200]

,날짜,시간,10,100,101,120,121,140,150,160,200,201,251,270,300,301,351,352,370,400,450,500,550,600,650,652,1000,1020,1040,1100,1200,1510,2510,3000,4510,5510,6000
3361,20200521,10,343667,107261,2942,21563,2881,1831,140808,7163,17779,10082,41336,8805,31776,20506,38214,74439,12688,44438,73441,112303,86261,18560,16709,11716,156349,9071,17838,33240,18792,7137,23833,2246,21369,14187,23894
3362,20200521,11,339887,99234,0,19127,0,0,140396,6696,17574,10795,28991,5549,34215,19625,40070,72799,13283,43235,76758,108672,82784,19970,17758,11140,146808,7853,17197,29479,18025,7938,22988,2045,20082,13671,22567
3363,20200521,12,322724,94243,0,16294,0,0,138321,6241,16659,9169,25061,5119,30711,17176,36812,66742,13299,38411,72243,101047,76343,20934,17346,10337,139759,6034,16643,27321,17244,7317,20898,2125,16320,11651,19487
3364,20200521,13,329873,93115,0,16674,0,0,137759,6276,16384,9279,27200,5424,27842,18269,37873,62039,12864,38264,70970,99667,78441,21897,18030,9923,148588,6709,17356,29011,18653,7241,21368,2206,18046,12433,19496
3365,20200521,14,334335,101138,7225,24275,6751,4802,149134,6716,17645,9427,55234,13798,30310,19473,39562,67227,15080,41066,73779,105869,84526,22942,19125,10239,158742,7698,17587,33825,19185,7900,21934,2359,18697,13151,21906
3366,20200521,15,347699,107829,12495,30297,10466,8501,153721,6923,18046,9987,75239,19689,32150,20722,40636,68661,14469,40978,73787,109283,86438,23215,19244,10018,165470,8308,17205,29813,19964,8259,22285,2296,18462,13428,21927
3367,20200521,16,357751,111307,12169,30732,11610,8625,155900,6859,19020,9696,78422,19635,33157,20736,42124,66022,15106,41366,79507,107175,89867,21101,17944,9823,177947,8682,17934,38517,20601,8028,25808,2849,20567,13913,23952


In [ ]:
# 잘 반영 됨
df_all3.loc[3361:3365]

,날짜,시간,10,100,101,120,121,140,150,160,200,201,251,270,300,301,351,352,370,400,450,500,550,600,650,652,1000,1020,1040,1100,1200,1510,2510,3000,4510,5510,6000
3361,20200521,10,343667,107261,12158,21563,10563,7479,140808,7163,17779,10082,41336,8805,31776,20506,38214,74439,12688,44438,73441,112303,86261,18560,16709,11716,156349,9071,17838,33240,18792,7137,23833,2246,21369,14187,23894
3362,20200521,11,339887,99234,11997,19127,9658,7347,140396,6696,17574,10795,28991,5549,34215,19625,40070,72799,13283,43235,76758,108672,82784,19970,17758,11140,146808,7853,17197,29479,18025,7938,22988,2045,20082,13671,22567
3363,20200521,12,322724,94243,10908,16294,9112,6607,138321,6241,16659,9169,25061,5119,30711,17176,36812,66742,13299,38411,72243,101047,76343,20934,17346,10337,139759,6034,16643,27321,17244,7317,20898,2125,16320,11651,19487
3364,20200521,13,329873,93115,11637,16674,9847,6771,137759,6276,16384,9279,27200,5424,27842,18269,37873,62039,12864,38264,70970,99667,78441,21897,18030,9923,148588,6709,17356,29011,18653,7241,21368,2206,18046,12433,19496
3365,20200521,14,334335,101138,12018,24275,10760,7948,149134,6716,17645,9427,55234,13798,30310,19473,39562,67227,15080,41066,73779,105869,84526,22942,19125,10239,158742,7698,17587,33825,19185,7900,21934,2359,18697,13151,21906


In [ ]:
# 2월29일(토13시~23시 총 11개 row) 결측치 넣어주기
df_all3[df_all3['날짜']==20200229]

,날짜,시간,10,100,101,120,121,140,150,160,200,201,251,270,300,301,351,352,370,400,450,500,550,600,650,652,1000,1020,1040,1100,1200,1510,2510,3000,4510,5510,6000
1416,20200229,0,70783,12481,1074,2920,1020,956,26231,677,1538,1264,12739,2632,4420,2020,5216,9242,1399,4006,17911,19245,6522,2564,1655,649,33515,613,2540,7873,5389,1026,3447,401,1706,1381,1872
1417,20200229,1,51754,8195,694,2141,617,703,19200,593,1224,1182,9213,2002,3347,1716,4172,6880,1096,3158,13492,14138,4941,1468,1075,567,22342,440,1545,5193,3925,866,2685,231,1240,1014,1428
1418,20200229,2,41031,7143,641,1936,597,595,14281,416,1127,935,7077,1872,3038,1710,3908,5830,913,2729,11316,11240,4552,1067,850,460,16738,372,1499,4047,2893,800,2313,237,1107,848,1307
1419,20200229,3,37688,7626,624,1742,420,500,13834,442,1249,1252,6565,1598,2881,1203,4085,5467,869,3477,10126,11589,4494,1109,785,586,14596,343,1424,3915,2283,636,2083,201,1257,807,1181
1420,20200229,4,46899,11199,781,2720,651,653,18429,609,1651,1236,7119,2281,3455,1668,4595,6909,1249,4582,11732,16410,5949,1820,1094,863,19631,442,2146,5882,2999,705,2737,391,1888,1163,1797
1421,20200229,5,80770,20795,1717,4251,1385,1018,38419,1110,2344,1422,11476,3601,4843,2867,6749,13788,3153,8670,15730,34010,11375,2411,1904,1527,54005,944,4123,13280,7195,912,3920,542,3485,2561,4000
1422,20200229,6,126497,35718,3163,6909,3872,1840,57764,1882,4335,1983,20971,4997,6889,4431,9901,24207,5710,14427,21414,55762,21321,4205,3587,3265,84433,2492,7496,19922,10239,1737,5548,895,7021,4730,8105
1423,20200229,7,156874,51431,4414,10409,5324,3191,72190,2940,6607,3434,31779,6462,8555,6317,14715,30339,5901,17341,26256,64036,33172,6197,6338,4227,88089,4516,10094,22121,11850,2424,7812,1407,10988,6980,11540
1424,20200229,8,189865,57387,5271,12740,5784,3666,100060,3474,7473,4234,39886,7702,9897,8178,18864,33339,6009,18842,30299,73396,36920,9958,8363,4464,94748,4062,10906,26327,15156,3405,10854,1697,11589,7241,12127
1425,20200229,9,229212,70651,7700,15575,7699,4894,110886,4191,9372,5591,51377,10538,13452,11898,24836,43816,8222,23023,38287,90570,45636,15229,10356,6143,118904,4950,12192,30064,16810,4651,13805,1932,11631,8396,14127


In [ ]:
df_all.loc[1141]

In [ ]:
col_list = list(df_all3.columns)[2:]

In [ ]:
# 2월29일(토13시~23시 총 11개 row) 결측치 넣어주기
data_0229_13_23 = list()
for i in range(11):
    data_0229_13_23.append(list(df_all3.loc[1586+i])[2:])
len(data_0229_13_23)

11

In [ ]:
# 2월29일(토13시~23시 총 11개 row) 결측치 넣어주기
list(df_all3.index)[-1] # 마지막 index num 확인 3446
for i in range(11): #2월 22일 (설날이라 데이터없음) & 3월 7일 데이터 넣어주기    3월7일 13시 loc[1586]
    df_all3.loc[3447+i] = [20200229, 13+i, *data_0229_13_23[i]]

In [ ]:
# 잘 추가 됨
df_all3.tail(12)

,날짜,시간,10,100,101,120,121,140,150,160,200,201,251,270,300,301,351,352,370,400,450,500,550,600,650,652,1000,1020,1040,1100,1200,1510,2510,3000,4510,5510,6000
3446,20200524,23,143609,26137,2242,6166,1609,1391,54633,1899,5182,2422,20343,5104,11331,5413,9455,22150,5370,10032,31124,34013,20205,6104,3782,2272,55574,1343,4686,8732,6986,2161,6487,410,3963,2686,4690
3447,20200229,13,288938,89088,11259,22241,10569,7311,149278,5796,17779,8007,66072,16281,25257,18140,35977,58884,12441,28812,51850,124194,63814,39867,23171,8608,156441,6417,14741,35428,18902,7948,19589,2753,13702,10433,17071
3448,20200229,14,292542,96386,11697,23272,11012,7454,170993,6027,18800,8243,69450,16744,27861,18972,36948,63177,13265,28521,53512,127030,68358,39881,23598,9418,162103,6339,15658,36894,19367,8022,20733,2809,13103,10600,18445
3449,20200229,15,300031,96822,11569,22888,10709,7500,185658,5857,18692,8335,69350,16181,27386,19899,38345,64866,13049,28670,54850,126570,68946,39359,24845,9125,168429,6257,15185,33174,20668,8977,21784,2971,13227,11404,19987
3450,20200229,16,306567,92925,11406,22646,10092,7306,182746,6121,19253,8218,67295,16102,27101,19383,36483,63798,12813,29201,53422,123759,68659,39886,24384,9414,173243,6574,15378,39613,21475,8924,22043,2869,13631,11647,19890
3451,20200229,17,299771,89966,9981,20994,9580,6810,152807,6075,19351,7473,62372,14388,24034,18647,34324,60906,12552,29227,49528,124589,66599,36644,22423,9541,171992,6779,13636,38989,21655,8117,20823,3012,14554,11800,19493
3452,20200229,18,245574,66152,7455,15905,7803,5559,129455,4502,15533,5872,50751,11399,18366,14322,26915,49698,10544,24267,40301,102730,53677,30313,17092,6306,144629,4690,10524,31870,18581,6296,16151,2322,10831,8181,14655
3453,20200229,19,192277,48772,5069,11569,5652,3599,99754,3388,10856,4110,38896,8655,13664,10969,19178,37995,7625,16345,32077,76443,38402,25804,11947,4622,113894,3238,8826,22974,14446,4791,11956,1615,7359,5966,9934
3454,20200229,20,164748,37809,3756,7876,4243,2737,83637,2727,7749,3361,31009,6355,9753,7903,14662,28191,6091,12096,26094,62687,28935,19927,8820,3441,100431,2652,7678,20471,13566,4287,9727,1191,5627,4442,7470
3455,20200229,21,141565,29185,2588,5538,3170,1907,65229,2208,5216,2738,24698,5039,7467,5767,10830,22784,4547,9166,20349,47693,21113,13681,6018,2409,89253,2002,6511,18068,12613,2922,7154,891,3958,3824,6045


In [ ]:
df_all3[df_all3['날짜']==20200406]  #2273
df_all.loc[2273]

In [ ]:
df_all3[df_all3['날짜']==20200323]  #1959
df_all.loc[1959]

In [ ]:
# 3월30일 작업(월02시~23시 22개)    #4월6일 데이터, 3월23일 데이터
data_0330_02_23 = list()
for i in range(22):
    data_0330_02_23.append( [int(i) for i in list((df_all3.loc[2273+i]+df_all3.loc[1959+i]/2))[2:]] )

In [ ]:
list(df_all3.index)[-1] # 마지막 index num 확인 3457
for i in range(22): #2월 22일 (설날이라 데이터없음) & 3월 7일 데이터 넣어주기    3월7일 13시 loc[1586]
    df_all3.loc[3458+i] = [20200330, 2+i, *data_0330_02_23[i]]

In [ ]:
df_all3.tail(24)

,날짜,시간,10,100,101,120,121,140,150,160,200,201,251,270,300,301,351,352,370,400,450,500,550,600,650,652,1000,1020,1040,1100,1200,1510,2510,3000,4510,5510,6000
3456,20200229,22,108967,20265,1988,3679,2285,1337,48963,1650,4024,1434,17369,3684,4925,3782,8030,15473,3371,5888,13732,31871,14055,9021,3698,2021,68466,1324,4693,14145,10806,1822,5326,599,3042,2676,3889
3457,20200229,23,73198,13065,1358,2504,1501,858,33330,1082,2368,977,12253,2489,3265,2366,5243,9131,1875,4127,9069,21149,8598,4908,2268,1196,43693,965,3044,9519,7751,1315,3083,305,1972,1767,2350
3458,20200330,2,54275,11712,981,2544,615,817,18941,639,2029,1971,8666,3244,4602,1866,4947,6861,1714,5018,13891,14505,7397,1611,1329,719,15939,438,2007,3655,2642,849,2656,219,1387,1284,1929
3459,20200330,3,64311,15280,856,3416,653,1031,23506,728,2346,2516,10743,4025,6059,2291,6069,8815,2191,6764,16045,17492,8801,1895,1601,1437,19325,483,2173,4382,2702,1163,3442,350,2042,1353,2614
3460,20200330,4,118591,28854,1828,6132,1240,1608,50597,1350,3234,3589,16951,6131,10094,4311,9554,18092,4226,12696,26035,36191,15610,2857,2466,2615,44544,906,4386,10414,5693,2001,6155,586,3897,2741,5289
3461,20200330,5,275559,61081,4631,12570,3823,3032,119798,3302,6816,4684,33802,9651,16525,9496,16807,47706,12628,28496,43601,95567,36867,5591,4978,5205,159596,2802,10839,32512,16510,3620,12599,1361,8969,7116,12564
3462,20200330,6,478888,120797,11038,22764,11120,6549,182919,6776,13888,7747,71897,15510,27704,20161,29813,86607,19093,46407,65200,146821,76166,11089,10507,12359,252300,8443,24136,58687,28555,6161,24730,3260,24200,16483,27673
3463,20200330,7,562571,165619,18771,34135,18831,11365,203772,10350,23348,11284,116377,21623,34755,31883,44176,109071,21175,61341,78856,174942,120090,15856,18114,17056,292155,17064,29982,69494,33201,8024,39897,6222,42112,25150,41110
3464,20200330,8,516063,150203,17628,36190,19879,11721,198083,10748,22006,11646,106880,22170,33501,27344,47412,101957,19940,55272,76198,166461,114329,16265,22122,14130,277856,13586,26856,65052,31494,7278,36267,6738,34537,23634,36431
3465,20200330,9,454044,128931,13878,32057,15981,9285,195943,9111,19739,11075,94764,20739,32284,23263,44939,93814,19371,52551,81516,150591,98011,17181,18724,13708,248309,10190,23455,56548,25432,7438,29564,3816,26422,18681,31319


In [ ]:
# 다시 날짜, 시간별로 sort 해주고 reset index
df_all4 = df_all3.sort_values(by=['날짜','시간']).reset_index(inplace = False, drop=True)
df_all4[df_all4['날짜']==20200330]

,날짜,시간,10,100,101,120,121,140,150,160,200,201,251,270,300,301,351,352,370,400,450,500,550,600,650,652,1000,1020,1040,1100,1200,1510,2510,3000,4510,5510,6000
1968,20200330,0,38243,7403,585,1695,426,351,13476,430,1304,820,6079,1633,2967,1271,2338,5138,1004,2456,9490,8930,3668,1344,903,563,13389,330,1257,2574,2011,521,1662,139,950,761,1130
1969,20200330,1,20378,3927,313,1060,315,223,7095,287,729,505,3129,1031,1716,692,1484,2760,685,1546,5564,4805,2026,680,508,289,6331,149,746,1220,1040,348,980,83,602,358,685
1970,20200330,2,54275,11712,981,2544,615,817,18941,639,2029,1971,8666,3244,4602,1866,4947,6861,1714,5018,13891,14505,7397,1611,1329,719,15939,438,2007,3655,2642,849,2656,219,1387,1284,1929
1971,20200330,3,64311,15280,856,3416,653,1031,23506,728,2346,2516,10743,4025,6059,2291,6069,8815,2191,6764,16045,17492,8801,1895,1601,1437,19325,483,2173,4382,2702,1163,3442,350,2042,1353,2614
1972,20200330,4,118591,28854,1828,6132,1240,1608,50597,1350,3234,3589,16951,6131,10094,4311,9554,18092,4226,12696,26035,36191,15610,2857,2466,2615,44544,906,4386,10414,5693,2001,6155,586,3897,2741,5289
1973,20200330,5,275559,61081,4631,12570,3823,3032,119798,3302,6816,4684,33802,9651,16525,9496,16807,47706,12628,28496,43601,95567,36867,5591,4978,5205,159596,2802,10839,32512,16510,3620,12599,1361,8969,7116,12564
1974,20200330,6,478888,120797,11038,22764,11120,6549,182919,6776,13888,7747,71897,15510,27704,20161,29813,86607,19093,46407,65200,146821,76166,11089,10507,12359,252300,8443,24136,58687,28555,6161,24730,3260,24200,16483,27673
1975,20200330,7,562571,165619,18771,34135,18831,11365,203772,10350,23348,11284,116377,21623,34755,31883,44176,109071,21175,61341,78856,174942,120090,15856,18114,17056,292155,17064,29982,69494,33201,8024,39897,6222,42112,25150,41110
1976,20200330,8,516063,150203,17628,36190,19879,11721,198083,10748,22006,11646,106880,22170,33501,27344,47412,101957,19940,55272,76198,166461,114329,16265,22122,14130,277856,13586,26856,65052,31494,7278,36267,6738,34537,23634,36431
1977,20200330,9,454044,128931,13878,32057,15981,9285,195943,9111,19739,11075,94764,20739,32284,23263,44939,93814,19371,52551,81516,150591,98011,17181,18724,13708,248309,10190,23455,56548,25432,7438,29564,3816,26422,18681,31319


In [ ]:
print(list(df_all4['날짜'].value_counts()))

[24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24]


In [ ]:
len(list(df_all4['날짜'].value_counts()))

138

In [ ]:
del_days = 5
df_all5 = df_all4.drop(df_all4.index[:del_days*24])
df_all6 = df_all5.sort_values(by=['날짜','시간']).reset_index(inplace = False, drop=True)

In [ ]:
df_all6

,날짜,시간,10,100,101,120,121,140,150,160,200,201,251,270,300,301,351,352,370,400,450,500,550,600,650,652,1000,1020,1040,1100,1200,1510,2510,3000,4510,5510,6000
0,20200106,0,78735,14431,1247,3825,996,831,26266,855,2384,1416,11584,2775,5778,2202,6123,10752,2655,4886,20080,21149,8022,2293,1595,1046,24230,739,2308,6055,3981,1263,3308,464,2539,1248,2239
1,20200106,1,51570,8678,743,2465,536,611,17079,614,1542,1203,6851,2155,3992,1621,4637,6753,1254,3698,14222,12945,5227,1171,1160,668,14505,403,1319,3391,2450,659,2143,300,1370,730,1438
2,20200106,2,40050,7673,611,2149,364,466,13957,492,1257,1231,5907,2327,3238,1374,3406,5142,882,3117,10997,10655,4200,829,885,591,10810,288,1365,3050,1903,817,1728,193,1133,682,1091
3,20200106,3,42615,9650,601,2178,325,590,15870,510,1439,1688,6961,2796,3924,1621,3756,5606,983,4356,11841,12577,5151,1156,942,773,12108,298,1411,3645,2037,906,2338,295,1321,760,1805
4,20200106,4,72065,18105,1034,4356,624,822,30432,1016,2555,2151,10481,4124,5567,2709,5916,10416,1953,7717,16488,24109,7969,1429,1445,1739,27955,649,2746,8651,3905,1360,3453,486,2726,1510,3439
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3187,20200524,19,314226,98345,10625,28618,8316,6684,141675,6619,23535,8572,67748,16791,33325,19124,42183,63229,13163,35383,73188,100849,84286,26458,15572,10341,139827,8254,16118,23304,14082,8447,21694,2180,15746,10903,21014
3188,20200524,20,300001,87871,8226,22706,6981,5743,142933,6295,20953,7851,61609,14119,30216,18610,35556,59070,12788,30675,68255,95107,76441,24140,13340,8872,142469,5225,15297,21919,14526,7332,19732,1990,14096,10028,17787
3189,20200524,21,304150,71126,6002,18317,4939,3779,133110,4781,15242,6101,48336,11534,25710,15651,27467,53787,12019,26317,61169,86434,61136,21076,9317,6392,138816,4072,12685,21135,14403,5443,16967,1359,11670,7963,14041
3190,20200524,22,236751,44947,3575,11455,3135,2536,98582,3267,8849,4553,33607,8069,18293,9723,16602,38615,8703,17694,47609,58492,37501,12090,5901,3938,100767,2489,8093,14427,10914,3861,11397,859,7270,5194,8230


In [ ]:
## 그냥 이대로 넣어주자

df_all6.to_csv('/content/drive/MyDrive/AI_team/eda_2/train.csv', index=False)

In [ ]:
# 3월30일 작업(월02시~23시 22개) 결측치 넣어주기

In [ ]:
# 날짜에 시간합쳐주고, 시간 컬럼 삭제

df_all6["날짜"] = df_all6["날짜"].astype(str) +'-'+ df_all6["시간"].astype(str)

# 시간 컬럼 삭제
df_all6

In [ ]:
df_all6

,날짜,10,100,101,120,121,140,150,160,200,201,251,270,300,301,351,352,370,400,450,500,550,600,650,652,1000,1020,1040,1100,1200,1510,2510,3000,4510,5510,6000
0,20200101,83247,19128,2611,5161,1588,892,32263,1636,7938,1894,14957,3527,11174,2930,8640,14175,2035,5231,18492,48520,15393,16628,8804,2790,38657,1311,3482,11299,7072,1176,3810,748,3920,2133,3799
1,20200101,89309,19027,3337,5502,1650,1043,35609,1644,9896,1967,14682,3420,13884,3062,8349,15654,2417,5483,15283,57904,15834,19785,8483,3381,45759,1162,3849,13180,8771,1283,3763,782,3483,2057,4010
2,20200101,66611,14710,2970,4631,1044,921,26821,1104,9343,1838,11120,3587,17141,3255,7025,12520,1859,5058,14327,53876,13866,24501,8167,2670,27425,768,2299,7986,5426,1536,3229,491,2634,1526,3388
3,20200101,53290,13753,2270,4242,1021,790,21322,909,10009,1630,9836,3842,20715,2661,7226,9821,1724,4863,12787,51615,13528,28695,9459,2325,18893,632,1716,5703,3156,1104,2882,431,2488,1268,3686
4,20200101,52095,17615,2406,3689,1840,922,22711,1354,15497,1264,10668,4559,28308,2894,8201,9038,960,5166,11479,47424,18156,26977,10944,3583,18135,875,2421,5816,2933,1206,2433,499,2952,1927,5608
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3307,20200524,314226,98345,10625,28618,8316,6684,141675,6619,23535,8572,67748,16791,33325,19124,42183,63229,13163,35383,73188,100849,84286,26458,15572,10341,139827,8254,16118,23304,14082,8447,21694,2180,15746,10903,21014
3308,20200524,300001,87871,8226,22706,6981,5743,142933,6295,20953,7851,61609,14119,30216,18610,35556,59070,12788,30675,68255,95107,76441,24140,13340,8872,142469,5225,15297,21919,14526,7332,19732,1990,14096,10028,17787
3309,20200524,304150,71126,6002,18317,4939,3779,133110,4781,15242,6101,48336,11534,25710,15651,27467,53787,12019,26317,61169,86434,61136,21076,9317,6392,138816,4072,12685,21135,14403,5443,16967,1359,11670,7963,14041
3310,20200524,236751,44947,3575,11455,3135,2536,98582,3267,8849,4553,33607,8069,18293,9723,16602,38615,8703,17694,47609,58492,37501,12090,5901,3938,100767,2489,8093,14427,10914,3861,11397,859,7270,5194,8230


In [ ]:
# print(df_all4)
df_all6['날짜'] = pd.to_datetime(df_all6['날짜'], format='%Y%m%d-%H', errors='raise')
df_all6.dtypes
# 날짜 타입으로 변경
df_all6['날짜'] = df_all6['날짜'].astype(str)

In [ ]:
print(*road_list)

10 100 101 120 121 140 150 160 200 201 251 270 300 301 351 352 370 400 450 500 550 600 650 652 1000 1020 1040 1100 1200 1510 2510 3000 4510 5510 6000


In [ ]:
# 컬럼 변경해주기
df_all6.columns = ['ds', *road_list]

In [ ]:
df_all6.head()

,ds,10,100,101,120,121,140,150,160,200,201,251,270,300,301,351,352,370,400,450,500,550,600,650,652,1000,1020,1040,1100,1200,1510,2510,3000,4510,5510,6000
120,20200106,78735,14431,1247,3825,996,831,26266,855,2384,1416,11584,2775,5778,2202,6123,10752,2655,4886,20080,21149,8022,2293,1595,1046,24230,739,2308,6055,3981,1263,3308,464,2539,1248,2239
121,20200106,51570,8678,743,2465,536,611,17079,614,1542,1203,6851,2155,3992,1621,4637,6753,1254,3698,14222,12945,5227,1171,1160,668,14505,403,1319,3391,2450,659,2143,300,1370,730,1438
122,20200106,40050,7673,611,2149,364,466,13957,492,1257,1231,5907,2327,3238,1374,3406,5142,882,3117,10997,10655,4200,829,885,591,10810,288,1365,3050,1903,817,1728,193,1133,682,1091
123,20200106,42615,9650,601,2178,325,590,15870,510,1439,1688,6961,2796,3924,1621,3756,5606,983,4356,11841,12577,5151,1156,942,773,12108,298,1411,3645,2037,906,2338,295,1321,760,1805
124,20200106,72065,18105,1034,4356,624,822,30432,1016,2555,2151,10481,4124,5567,2709,5916,10416,1953,7717,16488,24109,7969,1429,1445,1739,27955,649,2746,8651,3905,1360,3453,486,2726,1510,3439


In [ ]:
df_all6.to_csv('/content/drive/MyDrive/AI_team/eda/train_valid_merge_7days.csv', index=False)

7일 주기 train valid merge data

In [ ]:
df_all6 = pd.read_csv('/content/drive/MyDrive/AI_team/eda/train_valid_merge.csv')

In [ ]:
df_all6

In [ ]:
del_days=5
df_all7 = df_all6.drop(df_all6.index[:del_days*24])

In [ ]:
df_all7[df_all7['ds']=='2020-01-20 00:00:00']

,ds,10,100,101,120,121,140,150,160,200,201,251,270,300,301,351,352,370,400,450,500,550,600,650,652,1000,1020,1040,1100,1200,1510,2510,3000,4510,5510,6000
456,2020-01-20 00:00:00,87157,14952,1258,4098,992,940,26931,1052,2848,1591,13005,2984,5394,2429,5771,11749,2051,5465,19687,19841,8502,2192,1922,1159,23971,732,2375,5722,2730,1086,3755,518,2303,1330,2431


In [ ]:
df_all7

,ds,10,100,101,120,121,140,150,160,200,201,251,270,300,301,351,352,370,400,450,500,550,600,650,652,1000,1020,1040,1100,1200,1510,2510,3000,4510,5510,6000
120,2020-01-06 00:00:00,78735,14431,1247,3825,996,831,26266,855,2384,1416,11584,2775,5778,2202,6123,10752,2655,4886,20080,21149,8022,2293,1595,1046,24230,739,2308,6055,3981,1263,3308,464,2539,1248,2239
121,2020-01-06 01:00:00,51570,8678,743,2465,536,611,17079,614,1542,1203,6851,2155,3992,1621,4637,6753,1254,3698,14222,12945,5227,1171,1160,668,14505,403,1319,3391,2450,659,2143,300,1370,730,1438
122,2020-01-06 02:00:00,40050,7673,611,2149,364,466,13957,492,1257,1231,5907,2327,3238,1374,3406,5142,882,3117,10997,10655,4200,829,885,591,10810,288,1365,3050,1903,817,1728,193,1133,682,1091
123,2020-01-06 03:00:00,42615,9650,601,2178,325,590,15870,510,1439,1688,6961,2796,3924,1621,3756,5606,983,4356,11841,12577,5151,1156,942,773,12108,298,1411,3645,2037,906,2338,295,1321,760,1805
124,2020-01-06 04:00:00,72065,18105,1034,4356,624,822,30432,1016,2555,2151,10481,4124,5567,2709,5916,10416,1953,7717,16488,24109,7969,1429,1445,1739,27955,649,2746,8651,3905,1360,3453,486,2726,1510,3439
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3346,2020-05-24 19:00:00,314226,98345,10625,28618,8316,6684,141675,6619,23535,8572,67748,16791,33325,19124,42183,63229,13163,35383,73188,100849,84286,26458,15572,10341,139827,8254,16118,23304,14082,8447,21694,2180,15746,10903,21014
3347,2020-05-24 20:00:00,300001,87871,8226,22706,6981,5743,142933,6295,20953,7851,61609,14119,30216,18610,35556,59070,12788,30675,68255,95107,76441,24140,13340,8872,142469,5225,15297,21919,14526,7332,19732,1990,14096,10028,17787
3348,2020-05-24 21:00:00,304150,71126,6002,18317,4939,3779,133110,4781,15242,6101,48336,11534,25710,15651,27467,53787,12019,26317,61169,86434,61136,21076,9317,6392,138816,4072,12685,21135,14403,5443,16967,1359,11670,7963,14041
3349,2020-05-24 22:00:00,236751,44947,3575,11455,3135,2536,98582,3267,8849,4553,33607,8069,18293,9723,16602,38615,8703,17694,47609,58492,37501,12090,5901,3938,100767,2489,8093,14427,10914,3861,11397,859,7270,5194,8230


## AI가 아닌 기존 data에 가중치를 주며 답안 제출 

In [ ]:
df_all8 = df_all7.reset_index(inplace = False, drop=True)
df_all8.to_csv('/content/drive/MyDrive/AI_team/eda_2/train.csv', index=False)

In [ ]:
df_all7[456:456+24*2]

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/AI_team/eda_2/train.csv')

In [ ]:
a = df[3231-2*24*7:3231-24*7] #5월 2째 주 가중치 

In [ ]:
b = df[3231-24*7:3231]  #5월 3째 주 가중치 

In [ ]:
col_list = list(a.columns)[1:]
print(col_list)

['10', '100', '101', '120', '121', '140', '150', '160', '200', '201', '251', '270', '300', '301', '351', '352', '370', '400', '450', '500', '550', '600', '650', '652', '1000', '1020', '1040', '1100', '1200', '1510', '2510', '3000', '4510', '5510', '6000']


In [ ]:
Second_week_data = a.loc[:, col_list]*1.05
Second_week_data = Second_week_data.reset_index(inplace = False, drop=True)
Second_week_data

,10,100,101,120,121,140,150,160,200,201,251,270,300,301,351,352,370,400,450,500,550,600,650,652,1000,1020,1040,1100,1200,1510,2510,3000,4510,5510,6000
0,81866.40,15150.45,1294.65,4222.05,1030.05,925.05,30105.60,1117.20,3001.95,1626.45,12637.80,3386.25,7690.20,2928.45,6202.35,11651.85,2773.05,5665.80,21744.45,21826.35,8850.45,3343.20,2157.75,1501.50,29187.90,668.85,2734.20,5500.95,4376.40,1212.75,3775.80,353.85,2375.10,1688.40,2453.85
1,51112.95,9592.80,864.15,2748.90,686.70,600.60,18608.10,705.60,2037.00,1189.65,8731.80,2200.80,4644.15,1866.90,4229.40,7460.25,1861.65,3708.60,13818.00,13686.75,5731.95,1828.05,1291.50,901.95,16952.25,370.65,1963.50,3526.95,2685.90,1052.10,2264.85,269.85,1496.25,1068.90,1900.50
2,35461.65,8608.95,606.90,2297.40,411.60,527.10,15187.20,607.95,1496.25,1271.55,6954.15,2276.40,3877.65,1724.10,3544.80,5554.50,1331.40,3239.25,10895.85,11000.85,4840.50,1123.50,1015.35,789.60,12951.75,362.25,1573.95,2778.30,2123.10,919.80,2056.95,243.60,1212.75,973.35,1606.50
3,43586.55,10485.30,762.30,2957.85,582.75,678.30,18682.65,682.50,1761.90,1835.40,8249.85,2722.65,4596.90,1718.85,4608.45,6641.25,1724.10,4182.15,11820.90,13664.70,5299.35,1353.45,1079.40,1185.45,14898.45,409.50,1816.50,3567.90,2065.35,957.60,2585.10,295.05,1550.85,1006.95,1976.10
4,82614.00,20484.45,1536.15,4956.00,866.25,1142.40,36881.25,1046.85,2657.55,2419.20,12806.85,4213.65,7385.70,3367.35,6839.70,14592.90,3344.25,8700.30,18939.90,28668.15,9588.60,2312.10,1678.95,1689.45,34624.80,712.95,3105.90,7737.45,4326.00,1647.45,4796.40,605.85,3312.75,1964.55,3838.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,327313.35,106349.25,10589.25,32168.85,10563.00,9186.45,156381.75,7141.05,27406.05,9838.50,77877.45,18652.20,36450.75,20796.30,44870.70,66551.10,14836.50,36609.30,75048.75,101691.45,83107.50,26364.45,15414.00,10988.25,134761.20,7062.30,16202.55,26876.85,15006.60,9765.00,23349.90,3975.30,17782.80,11265.45,21203.70
164,320621.70,95997.30,9037.35,27322.05,8499.75,7557.90,143328.15,6454.35,21796.95,8853.60,72369.15,15322.65,31792.95,18515.70,39544.05,60705.75,13707.75,33540.15,68646.90,96721.80,75997.95,24323.25,12472.95,9591.75,138266.10,5776.05,16146.90,25894.05,15484.35,8650.95,21634.20,3363.15,15768.90,10255.35,18860.10
165,321308.40,78868.65,6641.25,20929.65,5996.55,4718.70,135882.60,5390.70,15220.80,7075.95,57177.75,12596.85,28399.35,16740.15,30480.45,57358.35,12906.60,28795.20,63730.80,87809.40,60759.30,17460.45,9001.65,6812.40,134217.30,4426.80,13185.90,23920.05,14785.05,6711.60,18833.85,2569.35,13023.15,8216.25,14732.55
166,249319.35,51972.90,4419.45,12752.25,4085.55,2853.90,101532.90,3702.30,9831.15,4770.15,37398.90,8395.80,20451.90,10510.50,19596.15,41678.70,9258.90,18658.50,48282.15,60187.05,37012.50,11299.05,5597.55,4314.45,99002.40,2706.90,9313.50,17472.00,11619.30,4648.35,12552.75,1569.75,7882.35,5656.35,9333.45


In [ ]:
Third_week_data = b.loc[:, col_list]*1.02
Third_week_data = Third_week_data.reset_index(inplace = False, drop=True)
Third_week_data

,10,100,101,120,121,140,150,160,200,201,251,270,300,301,351,352,370,400,450,500,550,600,650,652,1000,1020,1040,1100,1200,1510,2510,3000,4510,5510,6000
0,83706.30,15475.44,1530.00,3359.88,1107.72,981.24,29509.62,1125.06,3074.28,1651.38,12847.92,3121.20,7530.66,3017.16,5945.58,11522.94,2947.80,5807.88,19944.06,19297.38,10105.14,2970.24,1921.68,1134.24,28752.78,630.36,2845.80,5249.94,4417.62,1355.58,3738.30,412.08,2286.84,1651.38,2360.28
1,52272.96,10036.80,829.26,2403.12,709.92,556.92,18245.76,734.40,1875.78,1532.04,7895.82,2174.64,4885.80,1777.86,4381.92,6916.62,1787.04,3724.02,12703.08,12085.98,6417.84,1785.00,1140.36,717.06,17016.66,438.60,1901.28,3334.38,2612.22,939.42,2122.62,277.44,1417.80,1023.06,1801.32
2,39806.52,8051.88,775.20,2461.26,416.16,559.98,13624.14,507.96,1387.20,1257.66,6604.50,2139.96,4044.30,1371.90,4036.14,4857.24,1275.00,3262.98,9832.80,9579.84,5060.22,1126.08,1024.08,494.70,12172.68,328.44,1339.26,2820.30,1969.62,938.40,1799.28,232.56,1158.72,940.44,1335.18
3,41812.86,10339.74,795.60,2755.02,428.40,755.82,15854.88,542.64,1704.42,1836.00,7874.40,2175.66,4164.66,1531.02,4071.84,6326.04,1535.10,4548.18,11018.04,11897.28,6135.30,1154.64,1082.22,868.02,13727.16,332.52,1801.32,3386.40,2101.20,909.84,2495.94,343.74,1524.90,994.50,1950.24
4,79420.26,19995.06,1301.52,5119.38,987.36,1183.20,32743.02,987.36,2625.48,2431.68,12258.36,3572.04,6558.60,2939.64,6076.14,12226.74,3239.52,9235.08,16819.80,23948.58,10889.52,1983.90,1550.40,1448.40,32331.96,682.38,2972.28,7125.72,3989.22,1395.36,4467.60,523.26,2998.80,1793.16,3701.58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,320510.52,100311.90,10837.50,29190.36,8482.32,6817.68,144508.50,6751.38,24005.70,8743.44,69102.96,17126.82,33991.50,19506.48,43026.66,64493.58,13426.26,36090.66,74651.76,102865.98,85971.72,26987.16,15883.44,10547.82,142623.54,8419.08,16440.36,23770.08,14363.64,8615.94,22127.88,2223.60,16060.92,11121.06,21434.28
164,306001.02,89628.42,8390.52,23160.12,7120.62,5857.86,145791.66,6420.90,21372.06,8008.02,62841.18,14401.38,30820.32,18982.20,36267.12,60251.40,13043.76,31288.50,69620.10,97009.14,77969.82,24622.80,13606.80,9049.44,145318.38,5329.50,15602.94,22357.38,14816.52,7478.64,20126.64,2029.80,14377.92,10228.56,18142.74
165,310233.00,72548.52,6122.04,18683.34,5037.78,3854.58,135772.20,4876.62,15546.84,6223.02,49302.72,11764.68,26224.20,15964.02,28016.34,54862.74,12259.38,26843.34,62392.38,88162.68,62358.72,21497.52,9503.34,6519.84,141592.32,4153.44,12938.70,21557.70,14691.06,5551.86,17306.34,1386.18,11903.40,8122.26,14321.82
166,241486.02,45845.94,3646.50,11684.10,3197.70,2586.72,100553.64,3332.34,9025.98,4644.06,34279.14,8230.38,18658.86,9917.46,16934.04,39387.30,8877.06,18047.88,48561.18,59661.84,38251.02,12331.80,6019.02,4016.76,102782.34,2538.78,8254.86,14715.54,11132.28,3938.22,11624.94,876.18,7415.40,5297.88,8394.60


In [ ]:
my_submission = (Second_week_data + Third_week_data) / 2
my_submission

,10,100,101,120,121,140,150,160,200,201,251,270,300,301,351,352,370,400,450,500,550,600,650,652,1000,1020,1040,1100,1200,1510,2510,3000,4510,5510,6000
0,82786.350,15312.945,1412.325,3790.965,1068.885,953.145,29807.610,1121.130,3038.115,1638.915,12742.860,3253.725,7610.430,2972.805,6073.965,11587.395,2860.425,5736.840,20844.255,20561.865,9477.795,3156.720,2039.715,1317.870,28970.340,649.605,2790.000,5375.445,4397.010,1284.165,3757.050,382.965,2330.970,1669.890,2407.065
1,51692.955,9814.800,846.705,2576.010,698.310,578.760,18426.930,720.000,1956.390,1360.845,8313.810,2187.720,4764.975,1822.380,4305.660,7188.435,1824.345,3716.310,13260.540,12886.365,6074.895,1806.525,1215.930,809.505,16984.455,404.625,1932.390,3430.665,2649.060,995.760,2193.735,273.645,1457.025,1045.980,1850.910
2,37634.085,8330.415,691.050,2379.330,413.880,543.540,14405.670,557.955,1441.725,1264.605,6779.325,2208.180,3960.975,1548.000,3790.470,5205.870,1303.200,3251.115,10364.325,10290.345,4950.360,1124.790,1019.715,642.150,12562.215,345.345,1456.605,2799.300,2046.360,929.100,1928.115,238.080,1185.735,956.895,1470.840
3,42699.705,10412.520,778.950,2856.435,505.575,717.060,17268.765,612.570,1733.160,1835.700,8062.125,2449.155,4380.780,1624.935,4340.145,6483.645,1629.600,4365.165,11419.470,12780.990,5717.325,1254.045,1080.810,1026.735,14312.805,371.010,1808.910,3477.150,2083.275,933.720,2540.520,319.395,1537.875,1000.725,1963.170
4,81017.130,20239.755,1418.835,5037.690,926.805,1162.800,34812.135,1017.105,2641.515,2425.440,12532.605,3892.845,6972.150,3153.495,6457.920,13409.820,3291.885,8967.690,17879.850,26308.365,10239.060,2148.000,1614.675,1568.925,33478.380,697.665,3039.090,7431.585,4157.610,1521.405,4632.000,564.555,3155.775,1878.855,3770.190
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,323911.935,103330.575,10713.375,30679.605,9522.660,8002.065,150445.125,6946.215,25705.875,9290.970,73490.205,17889.510,35221.125,20151.390,43948.680,65522.340,14131.380,36349.980,74850.255,102278.715,84539.610,26675.805,15648.720,10768.035,138692.370,7740.690,16321.455,25323.465,14685.120,9190.470,22738.890,3099.450,16921.860,11193.255,21318.990
164,313311.360,92812.860,8713.935,25241.085,7810.185,6707.880,144559.905,6437.625,21584.505,8430.810,67605.165,14862.015,31306.635,18748.950,37905.585,60478.575,13375.755,32414.325,69133.500,96865.470,76983.885,24473.025,13039.875,9320.595,141792.240,5552.775,15874.920,24125.715,15150.435,8064.795,20880.420,2696.475,15073.410,10241.955,18501.420
165,315770.700,75708.585,6381.645,19806.495,5517.165,4286.640,135827.400,5133.660,15383.820,6649.485,53240.235,12180.765,27311.775,16352.085,29248.395,56110.545,12582.990,27819.270,63061.590,87986.040,61559.010,19478.985,9252.495,6666.120,137904.810,4290.120,13062.300,22738.875,14738.055,6131.730,18070.095,1977.765,12463.275,8169.255,14527.185
166,245402.685,48909.420,4032.975,12218.175,3641.625,2720.310,101043.270,3517.320,9428.565,4707.105,35839.020,8313.090,19555.380,10213.980,18265.095,40533.000,9067.980,18353.190,48421.665,59924.445,37631.760,11815.425,5808.285,4165.605,100892.370,2622.840,8784.180,16093.770,11375.790,4293.285,12088.845,1222.965,7648.875,5477.115,8864.025


In [ ]:
my_submission = 0.93*my_submission

In [ ]:
my_submission.to_csv('/content/drive/MyDrive/AI_team/eda_2/my_weight.csv', index=False)

In [ ]:
team_submission = pd.read_csv('/content/drive/MyDrive/AI_team/eda_2/7_submission.csv')
team_submission = team_submission.drop(['timestamp'], axis=1, inplace=False)
team_submission

,10,100,101,120,121,140,150,160,200,201,251,270,300,301,351,352,370,400,450,500,550,600,650,652,1000,1020,1040,1100,1200,1510,2510,3000,4510,5510,6000
0,74502.728995,12748.923136,969.677589,3292.363388,839.167270,758.752780,25925.705013,934.584520,2153.416339,1415.670007,10634.203968,2656.740933,5790.333981,2441.085049,5342.881547,10503.734891,2364.606146,4585.650429,18763.226916,16723.409676,8647.023773,2020.556098,1599.562709,962.757319,30674.263028,585.334295,2842.010133,5038.599498,4723.130547,1215.162475,3440.222433,389.945410,2240.581299,1437.154666,2165.401735
1,49654.982993,8922.196478,712.559032,2432.279816,546.595861,606.246717,17379.336669,645.566382,1607.086105,1181.260337,7628.825347,2215.993094,4208.033944,1802.790160,3960.065726,7151.898712,1660.085401,3494.035762,13206.599798,11558.426621,6228.593794,1387.765573,1149.330420,703.767429,17791.028397,382.738368,1771.410151,3285.867473,2892.222416,962.178029,2415.733713,261.707920,1424.001669,982.073626,1566.135010
2,40539.296535,7700.995561,605.757528,2185.877011,417.119424,558.697983,14163.634760,523.840424,1394.470675,1187.775421,6464.836812,2072.234454,3647.673353,1568.055102,3559.031238,5793.525042,1419.766496,3373.403668,10971.493513,9433.888655,5462.992149,1134.998820,972.383662,648.684080,12968.884501,300.569306,1409.691581,2525.174859,2152.265150,861.612010,2108.342167,234.764536,1239.875886,833.418346,1390.913107
3,44315.671343,9314.839189,672.277957,2445.446891,426.680670,636.836351,15563.012654,540.756594,1498.629538,1469.683313,6834.628787,2349.470289,3884.027309,1708.331953,3902.857747,6336.129926,1461.937351,4130.967137,10847.603478,10583.966076,5680.096742,1153.297609,1023.874521,823.612013,13892.079480,321.289130,1507.787246,3061.475052,2216.953708,885.709507,2335.662482,274.782480,1403.062104,933.328135,1744.527210
4,75825.613942,17979.361314,1189.609580,4415.625461,773.317663,1027.528729,31598.547288,921.359813,2313.686182,2315.994377,10934.126219,3750.930808,6318.684338,2907.002593,6150.924906,12274.152526,2939.810747,8373.921314,16588.675061,21579.422131,10198.214516,1879.219356,1640.726401,1652.070783,30944.000084,575.570670,2695.508484,6548.698621,3817.247834,1349.763130,3931.568258,513.537194,2703.743261,1685.005356,3380.469140
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,305519.205210,90747.659446,10020.552035,25716.220208,8898.732964,6765.787432,140645.002944,6052.575892,20436.329527,8367.169085,68695.114422,14452.643209,29585.260083,18399.526780,38913.373223,62344.583992,12718.717691,33768.944617,67924.157634,98301.535341,65892.856661,20349.101694,12220.015543,9027.507109,126352.779041,6516.612037,14514.252142,22127.127795,13268.137336,7680.531278,21088.629032,2345.550366,14304.820474,9620.154234,18267.629016
164,303033.926308,79247.495218,8414.358814,21584.657773,7472.417652,5772.871803,142038.744608,5470.068477,16662.627124,7463.106019,63087.048687,13735.949738,27912.089165,17007.698925,33813.233772,61251.989057,13468.715579,31472.682084,65028.389592,96318.341465,63290.351575,19884.811174,11615.004168,7903.006120,134226.689685,4828.450169,13565.939953,22951.506105,14076.049369,6812.890120,19538.423176,2182.686894,12955.904286,8897.172230,15809.273851
165,280800.819055,63152.366288,6062.629627,16149.359445,5345.749771,4014.645818,126640.076808,4357.399277,12168.785500,5925.458422,49780.284536,10884.573381,23614.606398,13748.715688,26547.701290,54215.153464,11939.573935,25506.803574,57073.138418,85014.384077,50631.166631,16037.035554,8447.560122,5883.874527,129464.834312,3799.820960,11222.316743,22774.678307,13506.784287,5531.244025,15870.157982,1301.218417,10247.080175,7038.559609,12135.522551
166,222098.477813,43902.051533,3786.178763,10861.819019,3240.370762,2483.422765,92634.203420,3069.313660,8266.501951,4066.427702,34090.606749,7622.596220,17613.056195,9252.608056,18102.877116,36580.517500,8384.564277,16971.037393,45038.171993,5917

In [ ]:
subtract = team_submission - 0.93*my_submission
subtract

,10,100,101,120,121,140,150,160,200,201,251,270,300,301,351,352,370,400,450,500,550,600,650,652,1000,1020,1040,1100,1200,1510,2510,3000,4510,5510,6000
0,-2488.576505,-1492.115714,-343.784661,-233.234062,-154.895780,-127.672070,-1795.372287,-108.066380,-672.030611,-108.520943,-1216.655832,-369.223317,-1287.365919,-323.623601,-305.905903,-272.542459,-295.589104,-749.610771,-621.930234,-2399.124774,-167.325577,-915.193502,-297.372241,-262.861781,3731.846828,-18.798355,247.310133,39.435648,633.911247,20.889025,-53.834067,33.787960,72.779199,-115.843034,-73.168715
1,1580.534843,-205.567522,-74.876618,36.590516,-102.832439,67.999917,242.291769,-24.033618,-212.356595,-84.325513,-103.017953,181.413494,-223.392806,107.976760,-44.198074,466.654162,-36.555449,37.867462,874.297598,-425.892829,578.941444,-292.302677,18.515520,-49.072221,1995.485247,6.437118,-25.712549,95.349023,428.596616,36.121229,375.560163,7.218070,68.968419,9.312226,-155.211290
2,5539.597485,-46.290389,-36.918972,-26.899889,32.211024,53.205783,766.361660,4.942274,53.666425,11.692771,160.064562,18.627054,-36.033397,128.415102,33.894138,952.065942,207.790496,349.866718,1332.671263,-136.132195,859.157349,88.944120,24.048712,51.484580,1286.024551,-20.601544,55.048931,-78.174141,249.150350,-2.450990,315.195217,13.350136,137.142336,-56.494004,23.031907
3,4604.945693,-368.804411,-52.145543,-211.037659,-43.504080,-30.029449,-496.938796,-28.933506,-113.209262,-237.517687,-663.147463,71.756139,-190.098091,197.142403,-133.477103,306.340076,-53.590649,71.363687,227.496378,-1302.354624,362.984492,-12.964241,18.721221,-131.251537,581.170830,-23.750170,-174.499054,-172.274448,279.507958,17.349907,-27.021118,-22.254870,-27.161646,2.653885,-81.220890
4,479.683042,-843.610836,-129.906970,-269.426239,-88.610987,-53.875271,-776.738262,-24.547837,-142.922768,60.335177,-721.196431,130.584958,-165.415162,-25.747757,145.059306,-196.980074,-121.642303,33.969614,-39.585439,-2887.357319,675.888716,-118.420644,139.078651,192.970533,-190.893316,-73.257780,-130.845216,-362.675429,-49.329466,-65.143520,-376.191742,-11.498956,-231.127489,-62.329794,-125.807560
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,4281.105660,-5349.775304,57.113285,-2815.812442,42.659164,-676.133018,731.036694,-407.404058,-3470.134223,-273.433015,349.223772,-2184.601091,-3170.386167,-341.265920,-1958.899177,1408.807792,-423.465709,-36.536783,-1686.579516,3182.330391,-12728.980639,-4459.396956,-2333.294057,-986.765441,-2631.125059,-682.229663,-664.701008,-1423.694655,-389.024264,-866.605822,-58.538668,-536.938134,-1432.509326,-789.572916,-1559.031684
164,11654.361508,-7068.464582,310.399264,-1889.551277,208.945602,-465.456597,7598.032958,-516.922773,-3410.962526,-377.547281,214.245237,-85.724212,-1203.081385,-428.824575,-1438.960278,5006.914307,1029.263429,1327.359834,734.234592,6233.454365,-8304.661475,-2875.102076,-512.079582,-765.147230,2359.906485,-335.630581,-1197.735647,514.591155,-13.855181,-687.369230,119.632576,-325.034856,-1062.367014,-627.845920,-1397.046749
165,-12865.931945,-7256.617762,127.699777,-2270.680905,214.786321,28.070618,320.594808,-416.904523,-2138.167100,-258.562628,266.865986,-443.538069,-1785.344352,-1458.723362,-653.306060,2032.346614,237.393235,-365.117526,-1574.140282,3187.366877,-6618.712669,-2078.420496,-157.260228,-315.617073,1213.361012,-189.990640,-925.622257,1627.524557,-199.606863,-171.264875,-935.030368,-538.103033,-1343.765575,-558.847541,-1374.759499
166,-6126.019237,-1583.709067,35.512013,-501.083731,-146.340488,-46.465535,-1336.037680,-201.793940,-502.063499,-311.179948,760.318149,-108.577480,-573.447205,-246.393344,1116.338766,-1115.172500,-48.657123,-97.429307,6.023543,3441.535223,-647.260817,-219.606541,251.270990,3.505231,674.139615,-43.517739,-169.113381,-485.483589,-472.357103,-107.118459,-442.499948,-266.130415,-334.666453,-385.357578,-319.572244
